In [57]:
import pandas as pd
df2 = pd.read_csv("US500.csv")
df = df2.copy()
df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,1990-01-02,359.70,353.40,359.70,352.00,NaN,1.78%
1,1990-01-03,358.80,359.70,360.60,357.90,NaN,-0.25%
2,1990-01-04,355.70,358.80,358.80,352.90,NaN,-0.86%
3,1990-01-05,352.20,355.70,355.70,351.40,NaN,-0.98%
4,1990-01-08,353.80,352.20,354.20,350.50,NaN,0.45%


In [58]:
df = df.drop(columns=['Vol.'])

In [59]:
df.describe().T

,count,unique,top,freq
Date,6725,6725,1990-01-02,1
Price,6725,5139,448.10,7
Open,6725,5121,461.90,7
High,6725,5068,448.60,8
Low,6725,5128,447.70,8
Change %,6725,676,0.00%,66


In [60]:
df.isnull().sum()

Date        0
Price       0
Open        0
High        0
Low         0
Change %    0
dtype: int64

In [ ]:
# "Hac." sütununu dönüştürmek için fonksiyon
def convert_volume(value):
    value = value.replace(',', '.')  # Virgülü noktaya çevir
    if value.endswith('M'):
        return float(value[:-1]) * 1_000_000  # "M" ise milyonla çarp
    elif value.endswith('K'):
        return float(value[:-1]) * 1_000      # "K" ise binle çarp
    else:
        return float(value)                    # Kısaltma yoksa direkt float

# Fonksiyonu "Hac." sütununa uygulama
df['Hac.'] = df['Hac.'].apply(convert_volume)

In [61]:
df.head()

,Date,Price,Open,High,Low,Change %
0,1990-01-02,359.70,353.40,359.70,352.00,1.78%
1,1990-01-03,358.80,359.70,360.60,357.90,-0.25%
2,1990-01-04,355.70,358.80,358.80,352.90,-0.86%
3,1990-01-05,352.20,355.70,355.70,351.40,-0.98%
4,1990-01-08,353.80,352.20,354.20,350.50,0.45%


In [62]:
# "Tarih" sütununu datetime formatına çevirme
#df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df.head()

,Date,Price,Open,High,Low,Change %
0,1990-01-02,359.70,353.40,359.70,352.00,1.78%
1,1990-01-03,358.80,359.70,360.60,357.90,-0.25%
2,1990-01-04,355.70,358.80,358.80,352.90,-0.86%
3,1990-01-05,352.20,355.70,355.70,351.40,-0.98%
4,1990-01-08,353.80,352.20,354.20,350.50,0.45%


In [63]:
# Eksik verileri kontrol etme
print(df.isnull().sum())

# Eksik verileri bir önceki değerle doldurma
#df = df.fillna(method='ffill')

Date        0
Price       0
Open        0
High        0
Low         0
Change %    0
dtype: int64


In [64]:
# İlk birkaç satırı görüntüleme
print(df.head())

# Sütun tiplerini kontrol etme
print(df.dtypes)

        Date   Price    Open    High     Low Change %
0 1990-01-02  359.70  353.40  359.70  352.00    1.78%
1 1990-01-03  358.80  359.70  360.60  357.90   -0.25%
2 1990-01-04  355.70  358.80  358.80  352.90   -0.86%
3 1990-01-05  352.20  355.70  355.70  351.40   -0.98%
4 1990-01-08  353.80  352.20  354.20  350.50    0.45%
Date        datetime64[ns]
Price               object
Open                object
High                object
Low                 object
Change %            object
dtype: object


In [65]:
# "Fark" sütununu sayısal değere dönüştürme
df['Change %'] = df['Change %'].str.replace('%', '')      # % sembolünü kaldır
df['Change %'] = df['Change %'].str.replace(',', '.')     # Virgülü noktaya çevir
df['Change %'] = df['Change %'].astype(float)             # Float'a dönüştür
# Dönüşüm sonrası kontrol
print(df['Change %'].head())

0    1.78
1   -0.25
2   -0.86
3   -0.98
4    0.45
Name: Change %, dtype: float64


In [66]:
# Tarih sütununun zaten datetime64[ns] formatında olduğunu varsayıyoruz
# Fiyat sütunlarını sayısal formata dönüştürme
#for col in ['Price', 'Open', 'High', 'Low']:
    #df[col] = df[col].str.replace(',', '.').astype(float)

for col in ['Price', 'Open', 'High', 'Low']:
    # Binlik ayırıcı noktaları kaldır (örneğin "1.001.30" → "1001.30" değil, bu veride anlam bozulabilir ama yine deneyelim)
    df[col] = df[col].str.replace('.', '', regex=False)  # Önce tüm noktaları kaldır
    # Daha sonra ondalık ayırıcının da noktadan farklı olduğunu varsayarsak değiştirebilirsin
    df[col] = df[col].str.replace(',', '.', regex=False)  # Virgülü noktaya çevir
    df[col] = df[col].astype(float)

# Dönüşüm sonrası sütun tiplerini kontrol etme
print(df.dtypes)

Date        datetime64[ns]
Price              float64
Open               float64
High               float64
Low                float64
Change %           float64
dtype: object


In [67]:
# 10 günlük SMA ekleme
df['SMA_10'] = df['Price'].rolling(window=10).mean()

# 10 günlük EMA ekleme
df['EMA_10'] = df['Price'].ewm(span=10, adjust=False).mean()
df.head()

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10
0,1990-01-02,35970.0,35340.0,35970.0,35200.0,1.78,NaN,35970.000000
1,1990-01-03,35880.0,35970.0,36060.0,35790.0,-0.25,NaN,35953.636364
2,1990-01-04,35570.0,35880.0,35880.0,35290.0,-0.86,NaN,35883.884298
3,1990-01-05,35220.0,35570.0,35570.0,35140.0,-0.98,NaN,35763.178062
4,1990-01-08,35380.0,35220.0,35420.0,35050.0,0.45,NaN,35693.509323


In [68]:
# 10 günlük SMA ekleme
df['SMA_10'] = df['Price'].rolling(window=10).mean()

# 10 günlük EMA ekleme
df['EMA_10'] = df['Price'].ewm(span=10, adjust=False).mean()
df.head()

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10
0,1990-01-02,35970.0,35340.0,35970.0,35200.0,1.78,NaN,35970.000000
1,1990-01-03,35880.0,35970.0,36060.0,35790.0,-0.25,NaN,35953.636364
2,1990-01-04,35570.0,35880.0,35880.0,35290.0,-0.86,NaN,35883.884298
3,1990-01-05,35220.0,35570.0,35570.0,35140.0,-0.98,NaN,35763.178062
4,1990-01-08,35380.0,35220.0,35420.0,35050.0,0.45,NaN,35693.509323


In [28]:
df['SMA_10'] = df['SMA_10'].fillna(df['Price'].expanding().mean()) # ilk 10 gün ortalaması
df.head(20)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10
0,1990-01-02,35970.0,35340.0,35970.0,35200.0,1.78,35970.000000,35970.000000
1,1990-01-03,35880.0,35970.0,36060.0,35790.0,-0.25,35925.000000,35953.636364
2,1990-01-04,35570.0,35880.0,35880.0,35290.0,-0.86,35806.666667,35883.884298
3,1990-01-05,35220.0,35570.0,35570.0,35140.0,-0.98,35660.000000,35763.178062
4,1990-01-08,35380.0,35220.0,35420.0,35050.0,0.45,35604.000000,35693.509323
5,1990-01-09,34960.0,35380.0,35420.0,34960.0,-1.19,35496.666667,35560.143992
6,1990-01-10,34730.0,34960.0,34960.0,34430.0,-0.66,35387.142857,35409.208720
7,1990-01-11,34850.0,34730.0,35010.0,34730.0,0.35,35320.000000,35307.534408
8,1990-01-12,33990.0,34850.0,34850.0,33950.0,-2.47,35172.222222,35067.982697
9,1990-01-15,33700.0,33990.0,33990.0,33660.0,-0.85,35025.000000,34819.258570


In [69]:
from ta.momentum import RSIIndicator

# 14 günlük RSI ekleme
rsi = RSIIndicator(close=df['Price'], window=14)
df['RSI_14'] = rsi.rsi()

In [70]:
#df['RSI_14'] = df['RSI_14'].fillna(df['RSI_14'].mean())

# İlk 30 günün RSI_14 ortalamasını hesapla (NaN hariç)
mean_first_20 = df.loc[:20, 'RSI_14'].mean()

# İlk 30 günün NaN değerlerini bu ortalama ile doldur
df.loc[:29, 'RSI_14'] = df.loc[:29, 'RSI_14'].fillna(mean_first_20)
df.head(20)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10,RSI_14
0,1990-01-02,35970.0,35340.0,35970.0,35200.0,1.78,NaN,35970.000000,19.569183
1,1990-01-03,35880.0,35970.0,36060.0,35790.0,-0.25,NaN,35953.636364,19.569183
2,1990-01-04,35570.0,35880.0,35880.0,35290.0,-0.86,NaN,35883.884298,19.569183
3,1990-01-05,35220.0,35570.0,35570.0,35140.0,-0.98,NaN,35763.178062,19.569183
4,1990-01-08,35380.0,35220.0,35420.0,35050.0,0.45,NaN,35693.509323,19.569183
5,1990-01-09,34960.0,35380.0,35420.0,34960.0,-1.19,NaN,35560.143992,19.569183
6,1990-01-10,34730.0,34960.0,34960.0,34430.0,-0.66,NaN,35409.208720,19.569183
7,1990-01-11,34850.0,34730.0,35010.0,34730.0,0.35,NaN,35307.534408,19.569183
8,1990-01-12,33990.0,34850.0,34850.0,33950.0,-2.47,NaN,35067.982697,19.569183
9,1990-01-15,33700.0,33990.0,33990.0,33660.0,-0.85,35025.0,34819.258570,19.569183


In [71]:
df.dtypes

Date        datetime64[ns]
Price              float64
Open               float64
High               float64
Low                float64
Change %           float64
SMA_10             float64
EMA_10             float64
RSI_14             float64
dtype: object

In [72]:
from ta.trend import MACD

# MACD ekleme (12, 26, 9)
macd = MACD(close=df['Price'], window_slow=26, window_fast=12, window_sign=9)
df['MACD'] = macd.macd()          # MACD çizgisi
df['MACD_Signal'] = macd.macd_signal()  # Sinyal çizgisi
df['MACD_Diff'] = macd.macd_diff()      # Fark (histogram)

In [73]:
df.head(40)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,MACD_Diff
0,1990-01-02,35970.0,35340.0,35970.0,35200.0,1.78,NaN,35970.000000,19.569183,NaN,NaN,NaN
1,1990-01-03,35880.0,35970.0,36060.0,35790.0,-0.25,NaN,35953.636364,19.569183,NaN,NaN,NaN
2,1990-01-04,35570.0,35880.0,35880.0,35290.0,-0.86,NaN,35883.884298,19.569183,NaN,NaN,NaN
3,1990-01-05,35220.0,35570.0,35570.0,35140.0,-0.98,NaN,35763.178062,19.569183,NaN,NaN,NaN
4,1990-01-08,35380.0,35220.0,35420.0,35050.0,0.45,NaN,35693.509323,19.569183,NaN,NaN,NaN
5,1990-01-09,34960.0,35380.0,35420.0,34960.0,-1.19,NaN,35560.143992,19.569183,NaN,NaN,NaN
6,1990-01-10,34730.0,34960.0,34960.0,34430.0,-0.66,NaN,35409.208720,19.569183,NaN,NaN,NaN
7,1990-01-11,34850.0,34730.0,35010.0,34730.0,0.35,NaN,35307.534408,19.569183,NaN,NaN,NaN
8,1990-01-12,33990.0,34850.0,34850.0,33950.0,-2.47,NaN,35067.982697,19.569183,NaN,NaN,NaN
9,1990-01-15,33700.0,33990.0,33990.0,33660.0,-0.85,35025.0,34819.258570,19.569183,NaN,NaN,NaN


In [74]:
for col in ['MACD', 'MACD_Signal', 'MACD_Diff']: # İlk günki değer ile dolduruldu
    first_value = df[col].iloc[33]
    df[col] = df[col].fillna(first_value)
df.isnull().sum()

Date           0
Price          0
Open           0
High           0
Low            0
Change %       0
SMA_10         9
EMA_10         0
RSI_14         0
MACD           0
MACD_Signal    0
MACD_Diff      0
dtype: int64

In [75]:
df.dtypes

Date           datetime64[ns]
Price                 float64
Open                  float64
High                  float64
Low                   float64
Change %              float64
SMA_10                float64
EMA_10                float64
RSI_14                float64
MACD                  float64
MACD_Signal           float64
MACD_Diff             float64
dtype: object

In [76]:
df3 = df.copy() # her ihtimale karşı verileri koruma
df3.head(40)

,Date,Price,Open,High,Low,Change %,SMA_10,EMA_10,RSI_14,MACD,MACD_Signal,MACD_Diff
0,1990-01-02,35970.0,35340.0,35970.0,35200.0,1.78,NaN,35970.000000,19.569183,-281.411114,-405.736075,124.324961
1,1990-01-03,35880.0,35970.0,36060.0,35790.0,-0.25,NaN,35953.636364,19.569183,-281.411114,-405.736075,124.324961
2,1990-01-04,35570.0,35880.0,35880.0,35290.0,-0.86,NaN,35883.884298,19.569183,-281.411114,-405.736075,124.324961
3,1990-01-05,35220.0,35570.0,35570.0,35140.0,-0.98,NaN,35763.178062,19.569183,-281.411114,-405.736075,124.324961
4,1990-01-08,35380.0,35220.0,35420.0,35050.0,0.45,NaN,35693.509323,19.569183,-281.411114,-405.736075,124.324961
5,1990-01-09,34960.0,35380.0,35420.0,34960.0,-1.19,NaN,35560.143992,19.569183,-281.411114,-405.736075,124.324961
6,1990-01-10,34730.0,34960.0,34960.0,34430.0,-0.66,NaN,35409.208720,19.569183,-281.411114,-405.736075,124.324961
7,1990-01-11,34850.0,34730.0,35010.0,34730.0,0.35,NaN,35307.534408,19.569183,-281.411114,-405.736075,124.324961
8,1990-01-12,33990.0,34850.0,34850.0,33950.0,-2.47,NaN,35067.982697,19.569183,-281.411114,-405.736075,124.324961
9,1990-01-15,33700.0,33990.0,33990.0,33660.0,-0.85,35025.0,34819.258570,19.569183,-281.411114,-405.736075,124.324961


In [77]:
df3.to_csv('US_Veri_without_Normalization.csv',index=False)

In [55]:
from sklearn.preprocessing import StandardScaler

# Normalleştirilecek sütunlar
normalize_columns = ['Price', 'Open', 'High', 'Low', 'SMA_10', 'EMA_10', 'MACD', 'MACD_Signal', 'MACD_Diff']

# StandardScaler ile standartlaştırma
scaler = StandardScaler()
df3[normalize_columns] = scaler.fit_transform(df3[normalize_columns])

# Sonucu kontrol etme
print(df3.head())

        Date     Price      Open      High       Low  Change %    SMA_10  \
0 1990-01-02  0.415203  0.394784  0.430602  0.378495      1.78  0.423959   
1 1990-01-03  0.412386  0.414491  0.433437  0.396872     -0.25  0.422519   
2 1990-01-04  0.402687  0.411675  0.427766  0.381298     -0.86  0.418731   
3 1990-01-05  0.391735  0.401978  0.417999  0.376626     -0.98  0.414037   
4 1990-01-08  0.396742  0.391030  0.413274  0.373823      0.45  0.412244   

     EMA_10     RSI_14     MACD  MACD_Signal  MACD_Diff  
0  0.425231  19.569183 -0.08142    -0.135355   0.108216  
1  0.424706  19.569183 -0.08142    -0.135355   0.108216  
2  0.422466  19.569183 -0.08142    -0.135355   0.108216  
3  0.418591  19.569183 -0.08142    -0.135355   0.108216  
4  0.416354  19.569183 -0.08142    -0.135355   0.108216  


In [56]:
df3.to_csv("US_veri_Son.csv",index=False)